# Experiment 8

This notebook contains the experiment to show a **narrowcasting example**, which is reported on Figure 8 in the [paper](https://arxiv.org/abs/2004.07641).

The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

## 1. Define the experiment parameters

#### Import libs

In [1]:
import numpy as np
import pickle
import multiprocessing

In [2]:
from lib.mobilitysim import MobilitySimulator
from lib.dynamics import DiseaseModel
from lib.parallel import create_ParallelSummary_from_DiseaseModel
from lib.distributions import CovidDistributions
from lib.data import collect_data_from_df
from lib.measures import (MeasureList, Interval,
                          SocialDistancingForPositiveMeasure)
from lib.town_maps import MapIllustrator

#### Set the random seed for reproducibility

In [3]:
# Choose random seed
c = 0
# Set it
np.random.seed(c)
# Define prefix string used to save plots
runstr = f'run{c}_'  

#### Set the number of roll-outs to simulate

In [4]:
random_repeats = 1

#### Define the number of parallel workers used for simulations 

In [5]:
num_workers = multiprocessing.cpu_count() - 1

#### Set the time to simulate

In [6]:
days_present = 35  # Days since March 10 on April 12 (March 10 is time 0 for our experiments)
days_future = 12 * 7  # Projecting 12 weeks into the future
max_time_future = 24.0 * days_future  # in hours

#### Define population/mobility parameters

Import downsampled version of Tübingen (population downsampled by 20x, sites by 10x) that was used used to generate mobility traces on the fly for inference (sites, home locations, etc)

In [7]:
case_downsample = 10                       # Downsample of population
mob_settings = 'lib/tu_settings_20_10.pk'  # Mobility settings (should match `case_downsample`)
town_name = 'LK Tübingen'                  # Name of town to fetch case data

# See town-generator.ipynb for an example on how to create the settings
with open(mob_settings, 'rb') as fp:
    obj = pickle.load(fp)
mob = MobilitySimulator(**obj)

#### Load the case data

We adjust the number of cases to the down-scaling of the population

In [8]:
new_cases_ = collect_data_from_df(town_name, 'new', until=days_present)
resistant_cases_ = collect_data_from_df(town_name, 'recovered', until=days_present)
fatality_cases_ = collect_data_from_df(town_name, 'fatality', until=days_present)

# fatality rate per age group
num_age_groups = fatality_cases_.shape[1] 
fatality_rates_by_age = (fatality_cases_[-1, :] / \
    (new_cases_[-1, :] + fatality_cases_[-1, :] + resistant_cases_[-1, :]))
print('Empirical fatality rates per age group:  ', fatality_rates_by_age.tolist())

# Instantiate correct state transition distributions (estimated from in literature)
distributions = CovidDistributions(fatality_rates_by_age=fatality_rates_by_age)

# Scale down cases based on number of people in simulation
new_cases, resistant_cases, fatality_cases = (
    1/case_downsample * new_cases_, 
    1/case_downsample * resistant_cases_, 
    1/case_downsample * fatality_cases_)
new_cases, resistant_cases, fatality_cases = np.ceil(new_cases), np.ceil(resistant_cases), np.ceil(fatality_cases)

Data last updated at:  13.04.2020, 00:00 Uhr
Data last updated at:  13.04.2020, 00:00 Uhr
Data last updated at:  13.04.2020, 00:00 Uhr
Empirical fatality rates per age group:   [0.0, 0.0, 0.0, 0.001430615164520744, 0.00964630225080386, 0.11728395061728394]


#### Define initial seed count per state 

These numbers are based on the *downscaled* infection counts on April 12.

In [10]:
with open('summaries/'+f'ave_present_seeds_{c}.pk', 'rb') as logfile:
    present_seeds = pickle.load(logfile)

#### Set epidemic parameters

Inferred using Bayesian optimization.

In [11]:
beta = 1.1383
inferred_params = {
    'betas' : {'education': beta,'social': beta,'bus_stop': beta,'office': beta,'supermarket': beta}, # site infectivity by type
    'mu': 1.0
}

#### Define standard testing parameters

The parameters are the same used for inference.

In [12]:
def standard_testing(max_time):
    standard_testing_params = {
        'testing_t_window'    : [0.0, max_time], # in hours
        'testing_frequency'   : 24.0,     # in hours
        'test_reporting_lag'  : 48.0,     # in hours (actual and self-report delay)
        'tests_per_batch'     : 10,       # assume 300 tests/day in LK Tübingen
        'test_smart_delta'    : 24.0 * 3, # in hours
        'test_smart_duration' : 24.0 * 7, # in hours
        'test_smart_action'   : 'isolate', 
        'test_smart_num_contacts'   : 10, 
        'test_targets'        : 'isym',
        'test_queue_policy'   : 'fifo',
        'smart_tracing'       : None, 
    }
    return standard_testing_params

#### Define helper I/O functions to load/save the output of the experiment

In [13]:
def save_summary(summary, filename):
    with open('summaries/' + filename, 'wb') as fp:
        pickle.dump(summary, fp)
    
def load_summary(filename):
    with open('summaries/' + filename, 'rb') as fp:
        summary = pickle.load(fp)
    return summary

---

## 2. Run the simulations

Define a helper function to run general type of experiment, fixing all parameters above.


In [14]:
def run(tparam, measure_list, t, local_seeds, dynamic_tracing=False):

    # add standard measure of positives staying isolated
    measure_list +=  [
        SocialDistancingForPositiveMeasure(
            t_window=Interval(0.0, t), p_stay_home=1.0)
    ]
    measure_list = MeasureList(measure_list)

    # run simulations
    summary = launch_parallel_simulations(
        mob_settings, 
        distributions, 
        random_repeats, num_workers, 
        inferred_params, local_seeds, tparam, measure_list, 
        max_time=t, 
        num_people=mob.num_people, 
        num_sites=mob.num_sites, 
        site_loc=mob.site_loc, 
        home_loc=mob.home_loc, 
        dynamic_tracing=dynamic_tracing,
        verbose=False)
    return summary
    

Generate a mobility instance

In [15]:
%%time
with open('lib/tu_settings_20_10.pk', 'rb') as fp:
        kwargs = pickle.load(fp)
mob = MobilitySimulator(**kwargs)
mob.verbose = True
mob.simulate(max_time=max_time_future)

Simulate mobility for 2016.00 time units... Simulated 355814 visits.
Find contacts... 
CPU times: user 1min 3s, sys: 1.24 s, total: 1min 5s
Wall time: 1min 5s


Simulate the epidemic based on the given mobility

In [16]:
%%time
measure_list =  [
    SocialDistancingForPositiveMeasure(
        t_window=Interval(0.0, max_time_future), p_stay_home=1.0)
]
measure_list = MeasureList(measure_list)

sim = DiseaseModel(mob, distributions)

sim.launch_epidemic(
    params={
        'betas' : [1.1383] * 5
    },
    initial_counts=present_seeds,
    testing_params=standard_testing(max_time_future),
    measure_list=measure_list,
    verbose=True)

summary = create_ParallelSummary_from_DiseaseModel(sim)

t: 2016.00 | 2016.00 hrs (84 d)
[Reached max time: 2016h (84d)]
CPU times: user 1min 57s, sys: 15.1 s, total: 2min 12s
Wall time: 2min 14s


---

## 3. Plot the results

Import libs

In [18]:
from lib.town_maps import MapIllustrator

Reproduce Figure 8 in the paper.

In [19]:
# add mobility traces to summary file because needed for plotting 
summary.mob = [mob]

In [20]:
day_start = 21
day_duration = 7

#
bbox = (48.4900, 48.5485, 9.0224, 9.1061) # coordinate bounding box
ill = MapIllustrator()
ill.marker_radius = 10.0
map_name = 'narrowcasting_baseline_10'
map_time = 24.0 * 30

sitemap = ill.empirical_infection_probability_map(
    bbox=bbox, 
    site_loc=mob.site_loc, 
    site_type=mob.site_type, 
    site_dict=mob.site_dict, 
    map_name=map_name, 
    sim=summary, 
    t0=24.0 * day_start, 
    t1=24.0 * (day_start + day_duration), 
    delta=0.0, 
    r=0)

sitemap